<a href="https://colab.research.google.com/github/ShreyasKadiri/Machine_Learning/blob/main/Create_CNN_Model_and_Optimize_using_Keras_Tuner_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner
#Ref: https://keras-team.github.io/keras-tuner/

     |████████████████████████████████| 71kB 10.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=9a4b308ac8bca468bd56e8b1cb48e39d62ef97c0ed44a7f9ad27fe19269192e2
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=1bed12cf3be03d5647428578e0bd0755c186ec07b181e47b26140bc30c9da111
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.9113333225250244

Best val_accuracy So Far: 0.9136666655540466
Total elapsed time: 00h 02m 20s
INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        32288     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1769568   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,803,946
Trainable params: 1,803,946
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1263 - accuracy: 0.9529 - val_loss: 0.2652 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0928 - accuracy: 0.9654 - val_loss: 0.2814 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0556 - accuracy: 0.9808 - val_loss: 0.3392 - val_accuracy: 0.9125
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0426 - accuracy: 0.9840 - val_loss: 0.3842 - val_accuracy: 0.9143
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0299 - accuracy: 0.9896 - val_loss: 0.4345 - val_accuracy: 0.9167
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0218 - accuracy: 0.9927 - val_loss: 0.4765 - val_accuracy: 0.9108
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0203 - accuracy: 0.9934 - val_loss: 0.5462 - val_accuracy

In [ ]:
#Ref: https://sites.google.com/view/amarnath-r/keras-deep-learning-image-classification-a-simple-example
